<a href="https://colab.research.google.com/github/haruhiysd/QM-Group-Project/blob/main/quarterly_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install Basemap
!pip install ipyleaflet
!pip install geopandas folium ipywidgets openpyxl
!pip install folium ipywidgets pandas
import pandas as pd
import folium
import json
import geopandas as gpd
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from ipywidgets import interactive
from IPython.display import display
import matplotlib.pylab as pylab
from datetime import date

pylab.rcParams['figure.figsize'] = (10, 8)

In [ ]:
df=pd.read_csv('/merged df.csv')

In [ ]:
rename_dict = {
    '01 Andalucía': 'Andalucía',
    '02 Aragón': 'Aragón',
    '03 Asturias, Principado de': 'Principado de Asturias',
    '04 Balears, Illes': 'Illes Balears',
    '05 Canarias': 'Canarias',
    '06 Cantabria': 'Cantabria',
    '07 Castilla y León': 'Castilla y León',
    '08 Castilla - La Mancha': 'Castilla-La Mancha',
    '09 Cataluña': 'Cataluña',
    '10 Comunitat Valenciana': 'Comunitat Valenciana',
    '11 Extremadura': 'Extremadura',
    '12 Galicia': 'Galicia',
    '13 Madrid, Comunidad de': 'Comunidad de Madrid',
    '14 Murcia, Región de': 'Región de Murcia',
    '15 Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra',
    '16 País Vasco': 'País Vasco',
    '17 Rioja, La': 'La Rioja',
}

In [ ]:
df['Period'] = pd.to_datetime(df['Period'], dayfirst=True, errors='coerce')

df_filtered = df[
    (df['Period'] >= '2015-01-01') &
    (df['Period'] <= '2023-10-01')
].copy()

In [ ]:
df_filtered = df_filtered[[
        'Autonomous_community',
        'Period',
        'Monthly_number_of_tourists',
        'Unemployment_rate',
        'HPI',
        'total_waste_capita',
        'Total_mixed_waste',
        'Total_waste_no_CDW_and_other',
]].copy()

In [ ]:
df_filtered = df_filtered[df_filtered['Period'].dt.month.isin([1, 4, 7, 10])]

In [ ]:
df_filtered['Autonomous_community'] = (
    df_filtered['Autonomous_community']
    .replace(rename_dict)
)
df_filtered

,Autonomous_community,Period,Monthly_number_of_tourists,Unemployment_rate,HPI,total_waste_capita,Total_mixed_waste,Total_waste_no_CDW_and_other
576,Andalucía,2015-01-01,NaN,NaN,96.637,504.6,3734660.0,4132582.0
577,Aragón,2015-01-01,NaN,NaN,98.632,427.9,457726.0,532954.0
578,Principado de Asturias,2015-01-01,NaN,NaN,97.509,519.4,404159.0,519529.0
579,Illes Balears,2015-01-01,NaN,NaN,95.424,800.6,766736.0,892903.0
580,Canarias,2015-01-01,NaN,NaN,97.087,594.1,1144978.0,1253565.0
...,...,...,...,...,...,...,...,...
2468,Comunidad Foral de Navarra,2023-10-01,NaN,9.25,132.884,NaN,NaN,NaN
2469,País Vasco,2023-10-01,NaN,6.38,137.252,NaN,NaN,NaN
2470,La Rioja,2023-10-01,NaN,9.71,135.324,NaN,NaN,NaN
2471,Nacional,2023-10-01,8187022.0,11.80,148.690,NaN,NaN,NaN


In [ ]:
gdf_spain.rename(
    columns={
        'acom_name': 'Autonomous_community',
        'year': 'Period',
    },
    inplace=True
)

In [ ]:
gdf_spain = gdf_spain[['Autonomous_community', 'geometry']].copy()

In [ ]:
def create_choropleth_map(year_month, metric_column, legend_label, max_value=None, min_value=None):

    year = year_month.year
    month = year_month.month


    df_year = df_filtered[
        (df_filtered['Period'].dt.year == year) &
        (df_filtered['Period'].dt.month == month)
    ].copy()

    merged_gdf = gdf_spain.merge(
        df_year,
        on=['Autonomous_community', 'Period'],
        how='left'
    )
    merged_gdf[metric_column] = pd.to_numeric(
        merged_gdf[metric_column],
        errors='coerce'
    ).fillna(0)

    m = folium.Map(location=[40, -4], zoom_start=6)

    if min_value is not None and max_value is not None:
        step = (max_value - min_value) / 5
        bins = list(range(int(min_value), int(max_value) + 1, int(step)))
        merged_gdf[metric_column] = merged_gdf[metric_column].clip(
            lower=min_value, upper=max_value
        )
    elif max_value is not None:
        step = max_value / 5
        bins = list(range(0, int(max_value) + 1, int(step)))
        merged_gdf[metric_column] = merged_gdf[metric_column].clip(
            lower=0, upper=max_value
        )
    else:
        bins = 8
        geojson_data = merged_gdf.__geo_interface__

    folium.Choropleth(
        geo_data=geojson_data,
        data=merged_gdf,
        columns=['Autonomous_community', metric_column],
        key_on='feature.properties.Autonomous_community',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.5,
        highlight=True,
        threshold_scale=bins,
        legend_name=legend_label
    ).add_to(m)

    folium.GeoJson(
        merged_gdf,
        name='Regions',
        style_function=lambda x: {
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['Autonomous_community', metric_column],
            aliases=['Region', metric_column]
        )
    ).add_to(m)

    m.save("my_map.html")
    return m


In [ ]:
@widgets.interact(
    Period=widgets.SelectionSlider(
        options=[date(year, month, 1) for year in range(2015, 2024) for month in [1, 4, 7, 10]],
        description='Period:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
)
def show_international_all(Period=date(2015, 1, 1)):
    return create_choropleth_map(
        Period,
        metric_column='HPI',
        legend_label='HPI',
        max_value=200
    )

interactive(children=(SelectionSlider(continuous_update=False, description='Period:', options=(datetime.date(2…